In [1]:
# !pip3 install torch
# !pip3 install pretrainedmodel

In [2]:
import os
import argparse
from os.path import join
import cv2
import dlib
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image as pil_image
from tqdm import tqdm
from torchvision import transforms

In [3]:
import pandas as pd
import numpy as np

In [4]:
xception_default_data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
}

In [5]:
def get_boundingbox(face, width, height, scale=1.3, minsize=None):
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * scale)
    if minsize:
        if size_bb < minsize:
            size_bb = minsize
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb


def preprocess_image(image, cuda=True):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    preprocess = xception_default_data_transforms['test']
    preprocessed_image = preprocess(pil_image.fromarray(image))
    preprocessed_image = preprocessed_image.unsqueeze(0)
    if cuda:
        preprocessed_image = preprocessed_image.cuda()
    return preprocessed_image


def predict_with_model(image, model, post_function=nn.Softmax(dim=1),
                       cuda=True):
    # Preprocess
    preprocessed_image = preprocess_image(image, cuda)

    # Model prediction
    output = model(preprocessed_image)
    output = post_function(output)

    # Cast to desired
    _, prediction = torch.max(output, 1)    # argmax
    prediction = float(prediction.cpu().numpy())

    return int(prediction), output

In [14]:
def test_full_image_network(video_path, model, output_path,
                            start_frame=0, end_frame=None, cuda=True):
    """
    Reads a video and evaluates a subset of frames with the a detection network
    that takes in a full frame. Outputs are only given if a face is present
    and the face is highlighted using dlib.
    :param video_path: path to video file
    :param model_path: path to model file (should expect the full sized image)
    :param output_path: path where the output video is stored
    :param start_frame: first frame to evaluate
    :param end_frame: last frame to evaluate
    :param cuda: enable cuda
    :return:
    
    # Modified to take in the model file instead of model
    """
    #print('Starting: {}'.format(video_path))
    
    predictions = []
    outputs = []

    # Read and write
    reader = cv2.VideoCapture(video_path)

    video_fn = video_path.split('/')[-1].split('.')[0]+'.avi'
    os.makedirs(output_path, exist_ok=True)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    fps = reader.get(cv2.CAP_PROP_FPS)
    num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
    writer = None

    # Face detector
    face_detector = dlib.get_frontal_face_detector()

    # Load model
#     model, *_ = model_selection(modelname='xception', num_out_classes=2)
#     if model_path is not None:
#         model = torch.load(model_path)
#         print('Model found in {}'.format(model_path))
#     else:
#         print('No model found, initializing random model.')
#     if cuda:
#         model = model.cuda()

    # Text variables
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    thickness = 2
    font_scale = 1

    # Frame numbers and length of output video
    frame_num = 0
    assert start_frame < num_frames - 1
    end_frame = end_frame if end_frame else num_frames
    pbar = tqdm(total=end_frame-start_frame)

    while reader.isOpened():
        _, image = reader.read()
        if image is None:
            break
        frame_num += 1

        if frame_num < start_frame:
            continue
        pbar.update(1)

        # Image size
#         print('getting image size')
        height, width = image.shape[:2]

        # Init output writer
#         print('init output writer')
        if writer is None:
            writer = cv2.VideoWriter(join(output_path, video_fn), fourcc, fps,
                                     (height, width)[::-1])

        # 2. Detect with dlib
#         print('detect with dlib')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 1)
        if len(faces):
            # For now only take biggest face
            face = faces[0]

            # --- Prediction ---------------------------------------------------
            # Face crop with dlib and bounding box scale enlargement
            x, y, size = get_boundingbox(face, width, height)
            cropped_face = image[y:y+size, x:x+size]

            # Actual prediction using our model
            prediction, output = predict_with_model(cropped_face, model,
                                                    cuda=cuda)
            predictions.append(prediction)
            outputs.append(output)
            # ------------------------------------------------------------------

            # Text and bb
            x = face.left()
            y = face.top()
            w = face.right() - x
            h = face.bottom() - y
            label = 'fake' if prediction == 1 else 'real'
            color = (0, 255, 0) if prediction == 0 else (0, 0, 255)
            output_list = ['{0:.2f}'.format(float(x)) for x in
                           output.detach().cpu().numpy()[0]]
            cv2.putText(image, str(output_list)+'=>'+label, (x, y+h+30),
                        font_face, font_scale,
                        color, thickness, 2)
            # draw box over face
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

        if frame_num >= end_frame:
            break

        # Show
#         print('show result')
        # cv2.imshow('test', image)
#         cv2.waitKey(33)     # About 30 fps
        writer.write(image)
    pbar.close()
    if writer is not None:
        writer.release()
        #print('Finished! Output saved under {}'.format(output_path))
    else:
        pass
        #print('Input video file was empty')
    return predictions, outputs

In [7]:
# metadata = pd.read_json('../input/deepfake-detection-challenge/train_sample_videos/metadata.json').T

# def predict_model(video_fn, model,
#                   start_frame=0, end_frame=30,
#                   plot_every_x_frames = 5):
#     """
#     Given a video and model, starting frame and end frame.
#     Predict on all frames.
    
#     """
#     fn = video_fn.split('.')[0]
#     label = metadata.loc[video_fn]['label']
#     original = metadata.loc[video_fn]['original']
#     video_path = f'../input/deepfake-detection-challenge/train_sample_videos/{video_fn}'
#     output_path = './'
#     test_full_image_network(video_path, model, output_path, start_frame=0, end_frame=30, cuda=False)
#     # Read output
#     vidcap = cv2.VideoCapture(f'{fn}.avi')
#     success,image = vidcap.read()
#     count = 0
#     fig, axes = plt.subplots(3, 2, figsize=(20, 15))
#     axes = axes = axes.flatten()
#     i = 0
#     while success:
#         # Show every xth frame
#         if count % plot_every_x_frames == 0:

#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             axes[i].imshow(image)
#             axes[i].set_title(f'{fn} - frame {count} - true label: {label}')
#             axes[i].xaxis.set_visible(False)
#             axes[i].yaxis.set_visible(False)
#             i += 1
#         success,image = vidcap.read()
#         count += 1
#     plt.tight_layout()
#     plt.show()
#     return

In [8]:
# def video_file_frame_pred(video_path, model,
#                           start_frame=0, end_frame=300,
#                           cuda=True, n_frames=5):
#     """
#     Predict and give result as numpy array
#     """
#     pred_frames = [int(round(x)) for x in np.linspace(start_frame, end_frame, n_frames)]
#     predictions = []
#     outputs = []
#     # print('Starting: {}'.format(video_path))

#     # Read and write
#     video_path = f'../input/deepfake-detection-challenge/train_sample_videos/{video_path}'
#     reader = cv2.VideoCapture(video_path)

#     video_fn = video_path.split('/')[-1].split('.')[0]+'.avi'
#     fourcc = cv2.VideoWriter_fourcc(*'MJPG')
#     fps = reader.get(cv2.CAP_PROP_FPS)
#     num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
#     writer = None

#     # Face detector
#     face_detector = dlib.get_frontal_face_detector()

#     # Text variables
#     font_face = cv2.FONT_HERSHEY_SIMPLEX
#     thickness = 2
#     font_scale = 1

#     # Frame numbers and length of output video
#     frame_num = 0
#     assert start_frame < num_frames - 1
#     end_frame = end_frame if end_frame else num_frames
#     while reader.isOpened():
#         _, image = reader.read()
#         if image is None:
#             break
#         frame_num += 1
#         if frame_num in pred_frames:
#             height, width = image.shape[:2]
#             # 2. Detect with dlib
#             gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#             faces = face_detector(gray, 1)
#             if len(faces):
#                 # For now only take biggest face
#                 face = faces[0]
#                 # --- Prediction ---------------------------------------------------
#                 # Face crop with dlib and bounding box scale enlargement
#                 x, y, size = get_boundingbox(face, width, height)
#                 cropped_face = image[y:y+size, x:x+size]

#                 # Actual prediction using our model
#                 prediction, output = predict_with_model(cropped_face, model,
#                                                         cuda=cuda)
#                 predictions.append(prediction)
#                 outputs.append(output)
#                 # ------------------------------------------------------------------
#         if frame_num >= end_frame:
#             break
#     # Figure out how to do this with torch
#     preds_np = [x.detach().cpu().numpy()[0][1] for x in outputs]
#     if len(preds_np) == 0:
#         return predictions, outputs, 0.5, 0.5, 0.5
#     try:
#         mean_pred = np.mean(preds_np)
#     except:
#         # couldnt find faces
#         mean_pred = 0.5
#     min_pred = np.min(preds_np)
#     max_pred = np.max(preds_np)
#     return predictions, outputs, mean_pred, min_pred, max_pred

In [9]:
model_path = './all_raw.p'
model = torch.load(model_path, map_location=torch.device('cpu'))
# model = torch.load(model_path)
# model = model.cuda()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'network.models.TransferModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `to

In [10]:
# predict_model('aettqgevhz.mp4', model)

In [11]:
# video_file_frame_pred('aettqgevhz.mp4', model, n_frames=4, cuda=False)

In [12]:
# predictions, outputs, mean_pred, min_pred, max_pred = video_file_frame_pred('aettqgevhz.mp4', model, n_frames=4, cuda=False)

In [15]:
test_full_image_network(f'../input/deepfake-detection-challenge/train_sample_videos/aettqgevhz.mp4', model, './', start_frame=0, end_frame=10, cuda=False)

100%|██████████| 10/10 [00:17<00:00,  1.77s/it]


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [tensor([[9.9931e-01, 6.8923e-04]], grad_fn=<SoftmaxBackward>),
  tensor([[9.9930e-01, 6.9542e-04]], grad_fn=<SoftmaxBackward>),
  tensor([[9.9988e-01, 1.1859e-04]], grad_fn=<SoftmaxBackward>),
  tensor([[9.9999e-01, 1.2059e-05]], grad_fn=<SoftmaxBackward>),
  tensor([[9.9999e-01, 8.9372e-06]], grad_fn=<SoftmaxBackward>),
  tensor([[1.0000e+00, 1.7430e-06]], grad_fn=<SoftmaxBackward>),
  tensor([[1.0000e+00, 6.8130e-07]], grad_fn=<SoftmaxBackward>),
  tensor([[1.0000e+00, 2.3482e-07]], grad_fn=<SoftmaxBackward>),
  tensor([[1.0000e+00, 4.9894e-07]], grad_fn=<SoftmaxBackward>),
  tensor([[1.0000e+00, 2.2484e-06]], grad_fn=<SoftmaxBackward>)])